## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Franco
- Apellido: Lores
- Legajo: 1638210

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
#Importar DataSet
london=pd.read_csv('TP2_airbnb_london.csv')

#Verificar Correcta Carga
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [4]:
#Imprimir Dimensiones del DataSet
london=london.drop(columns=['Unnamed: 0'])
print(f'El DataSet posee ' +str(np.shape(london)[0]) + ' Filas y ' +str(np.shape(london)[1]) + ' Columnas.')

El DataSet posee 51827 Filas y 8 Columnas.


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [5]:
#Corroborar Cantidad Valores Nulos
london.isnull().sum()

neighbourhood                     0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [6]:
#Tipo Variable Room_Type
Var1_Var2 = london[["neighbourhood", "room_type"]]
Var1_Var2.dtypes

neighbourhood    object
room_type        object
dtype: object

##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [7]:
## Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(london['neighbourhood'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [8]:
neigh_dummies.head()

,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
neigh_dummies.shape

(51827, 33)

In [10]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies = pd.get_dummies(london['room_type'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [11]:
room_dummies.head()

,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


In [12]:
room_dummies.shape

(51827, 3)

Joinear las variables dummies al dataset



In [13]:
london=pd.concat([london,room_dummies,neigh_dummies],axis=1)

In [14]:
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,Haringey,Private room,35,1,131,1.71,4,262,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Ealing,Private room,70,2,2,0.07,6,62,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Islington,Private room,45,1,14,0.14,2,364,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295,1,0,...,0,0,0,0,0,0,1,0,0,0
4,Barnet,Private room,29,5,114,1.13,3,285,0,1,...,0,0,0,0,0,0,0,0,0,0


##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [15]:
#Reducir Numero de Filas para Rendimiento de la PC
london=london.iloc[:5000]
london.shape

(5000, 44)

In [16]:
#Definir atributos que seran la variable x e y
y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [17]:
#Dividir DataSet en Train y Test
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=0)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [18]:
#Visualizar DataSet Train.
X_train.head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,Shared room,Barking and Dagenham,Barnet,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
2858,1,230,4.01,1,345,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1559,2,2,0.09,1,231,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1441,2,17,0.32,4,221,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2179,5,32,0.52,3,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1390,2,36,0.54,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
#Separar Variables para Scaler
X_train1=X_train.iloc[:,0:5]
X_train1.head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
2858,1,230,4.01,1,345
1559,2,2,0.09,1,231
1441,2,17,0.32,4,221
2179,5,32,0.52,3,0
1390,2,36,0.54,1,0


In [20]:
#Separar Variables Dummies
X_train2=X_train.iloc[:,5:41]
X_train2.head()

,Entire home/apt,Private room,Shared room,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
2858,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1559,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1441,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2179,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1390,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [21]:
#Aplicar Scaler
scaler = preprocessing.StandardScaler().fit(X_train1)

In [22]:
xtrain_scal = scaler.transform(X_train1)
print(xtrain_scal.mean(axis=0))
print(xtrain_scal.std(axis=0))

[ 2.03012210e-17  6.09036631e-18  1.09119063e-17 -4.56777473e-18
 -9.23705556e-17]
[1. 1. 1. 1. 1.]


In [23]:
X_test.head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,Shared room,Barking and Dagenham,Barnet,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
398,2,10,0.13,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3833,1,65,1.24,6,10,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4836,1,13,7.50,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4572,4,52,1.06,1,365,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
636,1,32,0.42,2,342,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
#Separar Variables para Scaler
X_test1=X_test.iloc[:,0:5]
X_test1.head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
398,2,10,0.13,1,0
3833,1,65,1.24,6,10
4836,1,13,7.50,2,0
4572,4,52,1.06,1,365
636,1,32,0.42,2,342


In [ ]:
#Separar Variables Dummies
X_test2=X_test.iloc[:,5:41]
X_test2.head()

In [26]:
#Aplicar Scaler
xtest_scal = scaler.transform(X_test1)
print(xtest_scal.mean(axis=0))
print(xtest_scal.std(axis=0))

[ 0.0017758  -0.07269996 -0.08745025  0.0222788  -0.0396277 ]
[1.01823669 0.94723951 0.90993815 1.3853538  1.01408136]


In [27]:
#Volver a Unificar DataSets Train y Test
xtrain_scal=np.concatenate((xtrain_scal,X_train2),axis=1)
xtrain_scal.shape

(3500, 41)

In [28]:
xtest_scal=np.concatenate((xtest_scal,X_test2),axis=1)
xtrain_scal.shape

(3500, 41)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [30]:
#Crear Modelo Regresion Lineal y Calculo del error.
regli=LinearRegression()
regli.fit(xtrain_scal,Y_train)
Yprediction=regli.predict(xtest_scal)

In [31]:
(mean_squared_error(Y_test,Yprediction))

1333.297619913737

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [33]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
svc = svm.SVC()

In [34]:
#Definir Parametros y Ejecutar GridSearch
parameters = {'kernel':('linear','rbf'),
             'C':[1,10,100],
             'gamma':[0.0005,0.005,0.5]}

In [35]:
clf=GridSearchCV(svc,param_grid=parameters,refit=True,cv=5,verbose=3)
clf.fit(xtrain_scal, Y_train.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\Bangho\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5] END ..C=1, gamma=0.0005, kernel=linear;, score=0.060 total time=   3.6s
[CV 2/5] END ..C=1, gamma=0.0005, kernel=linear;, score=0.079 total time=   3.2s
[CV 3/5] END ..C=1, gamma=0.0005, kernel=linear;, score=0.073 total time=   3.2s
[CV 4/5] END ..C=1, gamma=0.0005, kernel=linear;, score=0.073 total time=   3.3s
[CV 5/5] END ..C=1, gamma=0.0005, kernel=linear;, score=0.071 total time=   3.8s
[CV 1/5] END .....C=1, gamma=0.0005, kernel=rbf;, score=0.039 total time=   3.2s
[CV 2/5] END .....C=1, gamma=0.0005, kernel=rbf;, score=0.040 total time=   2.9s
[CV 3/5] END .....C=1, gamma=0.0005, kernel=rbf;, score=0.040 total time=   2.9s
[CV 4/5] END .....C=1, gamma=0.0005, kernel=rbf;, score=0.040 total time=   3.0s
[CV 5/5] END .....C=1, gamma=0.0005, kernel=rbf;, score=0.039 total time=   3.6s
[CV 1/5] END ...C=1, gamma=0.005, kernel=linear;, score=0.060 total time=   3.3s
[CV 2/5] END ...C=1, gamma=0.005, kernel=linear;, score=0.079 total time=   3.5s
[CV 3/5] END ...C=1, gamma=0

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 10, 100], 'gamma': [0.0005, 0.005, 0.5],
                         'kernel': ('linear', 'rbf')},
             verbose=3)

In [36]:
scores=clf.cv_results_
scores_df=pd.DataFrame.from_dict(scores)
scores_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.324411,0.212989,1.214555,0.048418,1,0.0005,linear,"{'C': 1, 'gamma': 0.0005, 'kernel': 'linear'}",0.060000,0.078571,0.072857,0.072857,0.071429,0.071143,0.006088,1
1,1.719293,0.215415,1.500987,0.104151,1,0.0005,rbf,"{'C': 1, 'gamma': 0.0005, 'kernel': 'rbf'}",0.038571,0.040000,0.040000,0.040000,0.038571,0.039429,0.000700,18
2,2.324135,0.109423,1.366019,0.170327,1,0.005,linear,"{'C': 1, 'gamma': 0.005, 'kernel': 'linear'}",0.060000,0.078571,0.072857,0.072857,0.071429,0.071143,0.006088,1
3,1.944414,0.145597,1.401650,0.019636,1,0.005,rbf,"{'C': 1, 'gamma': 0.005, 'kernel': 'rbf'}",0.057143,0.061429,0.068571,0.065714,0.064286,0.063429,0.003897,14
4,2.209271,0.079915,1.251045,0.072793,1,0.5,linear,"{'C': 1, 'gamma': 0.5, 'kernel': 'linear'}",0.060000,0.078571,0.072857,0.072857,0.071429,0.071143,0.006088,1


##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [37]:
print("Los Mejores Parametros son %s con un score de %0.5f" % (clf.best_params_, clf.best_score_))

Los Mejores Parametros son {'C': 1, 'gamma': 0.0005, 'kernel': 'linear'} con un score de 0.07114


In [38]:
Yprediction2=clf.predict(xtest_scal)
print(Yprediction2)

[150  50 225 ...  45  45  35]


In [40]:
(mean_squared_error(Y_test,Yprediction2))

1664.934

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [47]:
parameters2 = {'kernel':('linear','rbf'),
             'C':[100,50,25],
             'gamma':[0.000005,1,2]}

In [48]:
clf=GridSearchCV(svc,param_grid=parameters2,refit=True,cv=5,verbose=3)
clf.fit(xtrain_scal, Y_train.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\Bangho\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5] END .C=100, gamma=5e-06, kernel=linear;, score=0.063 total time=  18.8s
[CV 2/5] END .C=100, gamma=5e-06, kernel=linear;, score=0.067 total time=  17.7s
[CV 3/5] END .C=100, gamma=5e-06, kernel=linear;, score=0.063 total time=  18.9s
[CV 4/5] END .C=100, gamma=5e-06, kernel=linear;, score=0.069 total time=  18.3s
[CV 5/5] END .C=100, gamma=5e-06, kernel=linear;, score=0.069 total time=  19.9s
[CV 1/5] END ....C=100, gamma=5e-06, kernel=rbf;, score=0.039 total time=   2.6s
[CV 2/5] END ....C=100, gamma=5e-06, kernel=rbf;, score=0.040 total time=   2.6s
[CV 3/5] END ....C=100, gamma=5e-06, kernel=rbf;, score=0.040 total time=   2.5s
[CV 4/5] END ....C=100, gamma=5e-06, kernel=rbf;, score=0.040 total time=   3.0s
[CV 5/5] END ....C=100, gamma=5e-06, kernel=rbf;, score=0.039 total time=   2.7s
[CV 1/5] END .....C=100, gamma=1, kernel=linear;, score=0.063 total time=  18.6s
[CV 2/5] END .....C=100, gamma=1, kernel=linear;, score=0.067 total time=  17.5s
[CV 3/5] END .....C=100, gam

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [100, 50, 25], 'gamma': [5e-06, 1, 2],
                         'kernel': ('linear', 'rbf')},
             verbose=3)

In [49]:
scores=clf.cv_results_
scores_df=pd.DataFrame.from_dict(scores)
scores_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,17.782863,0.750542,1.039908,0.059161,100,0.000005,linear,"{'C': 100, 'gamma': 5e-06, 'kernel': 'linear'}",0.062857,0.067143,0.062857,0.068571,0.068571,0.066000,0.002619,7
1,1.496894,0.041956,1.274693,0.151240,100,0.000005,rbf,"{'C': 100, 'gamma': 5e-06, 'kernel': 'rbf'}",0.038571,0.040000,0.040000,0.040000,0.038571,0.039429,0.000700,16
2,17.746224,0.938810,1.048049,0.087122,100,1,linear,"{'C': 100, 'gamma': 1, 'kernel': 'linear'}",0.062857,0.067143,0.062857,0.068571,0.068571,0.066000,0.002619,7
3,3.593498,0.048095,1.229943,0.043503,100,1,rbf,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}",0.054286,0.055714,0.041429,0.054286,0.072857,0.055714,0.010020,14
4,18.921373,2.242827,1.131145,0.134553,100,2,linear,"{'C': 100, 'gamma': 2, 'kernel': 'linear'}",0.062857,0.067143,0.062857,0.068571,0.068571,0.066000,0.002619,7


In [50]:
print("Los Mejores Parametros son %s con un score de %0.5f" % (clf.best_params_, clf.best_score_))

Los Mejores Parametros son {'C': 25, 'gamma': 5e-06, 'kernel': 'linear'} con un score de 0.06943


In [51]:
Yprediction3=clf.predict(xtest_scal)
print(Yprediction3)

[ 95  50 225 ...  45  45  30]


In [52]:
(mean_squared_error(Y_test,Yprediction3))

2091.887333333333